In [4]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


##**Sistema de Recomendação em R**

In [5]:
# Obs: Caso tenha problemas com a acentuação, consulte este link:
# https://support.rstudio.com/hc/en-us/articles/200532197-Character-Encoding

Passos que devem ser executados na construção do sistema de recomendação

1. Carregar e formatar dados.
2. Calcular a similaridade entre usuários. 
3. Prever das classificações desconhecidas para os usuários. 
4. Recomendar itens aos usuários com base na pontuação de similaridade de usuário.
---

### O objetivo deste sistema de recomendação é oferecer filmes para usuários com base em usuários semelhantes

In [6]:
%%R
# Pacotes
install.packages('reshape2')
install.packages('data.table')
install.packages('dplyr')

library(reshape2)
library(data.table)
library(dplyr)

# Esse pacote é uma junção dos pacotes data.table e dplyr
install.packages("dtplyr")
library(dtplyr)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/reshape2_1.4.4.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 37307 bytes (36 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write

###**Carregando os Dados**
-  Esse dataset contém as avaliações de 6 filmes dadas por 6 usuários diferentes. 
-  A escala de avaliação vai de 0 a 5
-  Nem todos os usuários deram a avaliação a todos os filmes

In [7]:
%%R
ratings = read.csv("movie_ratings.csv")

In [8]:
%%R
dim(ratings)

[1] 31  3


In [9]:
%%R
str(ratings)

'data.frame':	31 obs. of  3 variables:
 $ critic: Factor w/ 6 levels "Garrincha","Pele Silva",..: 2 2 2 2 2 5 5 5 5 5 ...
 $ title : Factor w/ 6 levels "Just My Luck",..: 2 3 6 4 5 2 3 1 4 6 ...
 $ rating: num  3 4 3.5 5 3 3 4 2 3 2 ...


In [10]:
%%R
# Levels mostra as categorias disponíveis nessa variável do tipo fator
levels(ratings$critic)

[1] "Garrincha"       "Pele Silva"      "Romario Moreira" "Ronaldo Tavares"
[5] "Zico Gomes"      "Zinedine Zidane"


In [11]:
%%R
levels(ratings$title)

[1] "Just My Luck"       "Lady in the Water"  "Snakes on a Plane" 
[4] "Superman Returns"   "The Night Listener" "You Me and Dupree" 


In [12]:
%%R
#View(ratings)

NULL


##**Formatando e processando os dados**
Necessário converter os dados em um formato de matriz, sendo:
- Usuário (critic) como colunas
- Files (title) como linhas
- Avaliaçãoes (ratings) como valores


In [13]:
%%R
# Função acast do pacote reshape2: transforma um conjunto de dados em umaa matriz
movie_ratings = as.data.frame(acast(ratings, title~critic, value.var = "rating"))
movie_ratings

                   Garrincha Pele Silva Romario Moreira Ronaldo Tavares
Just My Luck              NA         NA             3.0             3.0
Lady in the Water         NA        3.0              NA             2.5
Snakes on a Plane        4.5        4.0             3.5             3.5
Superman Returns         4.0        5.0             4.0             3.5
The Night Listener        NA        3.0             4.5             3.0
You Me and Dupree        1.0        3.5             2.5             2.5
                   Zico Gomes Zinedine Zidane
Just My Luck                2             1.5
Lady in the Water           3             3.0
Snakes on a Plane           4             3.5
Superman Returns            3             5.0
The Night Listener          3             3.0
You Me and Dupree           2             3.5


Podemos perceber que o usuário Garrincha ainda não avaliou filmes, pois provavelmente ele ainda não assistiu esses filmes. 

Nosso sistema de recomendação deve recomendar filmes para usuários como Garrincha, com base em usuários similares



---
##**Calculando a Similaridade**
Podemos usar medidas de distância como:

- distância euclidiana
- distância cosine 
- coeficiente de Pearson para correlação

O parâmetro "complete.obs" considera todas as observações no dataset

In [14]:
%%R
?cor
sim_users = cor(movie_ratings[,1:6], use = "complete.obs")
#View(sim_users)
head(sim_users)

                Garrincha Pele Silva Romario Moreira Ronaldo Tavares Zico Gomes
Garrincha       1.0000000  0.6628490       0.8934051       0.9912407  0.9244735
Pele Silva      0.6628490  1.0000000       0.9285714       0.7559289  0.3273268
Romario Moreira 0.8934051  0.9285714       1.0000000       0.9449112  0.6546537
Ronaldo Tavares 0.9912407  0.7559289       0.9449112       1.0000000  0.8660254
Zico Gomes      0.9244735  0.3273268       0.6546537       0.8660254  1.0000000
Zinedine Zidane 0.3812464  0.9449112       0.7559289       0.5000000  0.0000000
                Zinedine Zidane
Garrincha             0.3812464
Pele Silva            0.9449112
Romario Moreira       0.7559289
Ronaldo Tavares       0.5000000
Zico Gomes            0.0000000
Zinedine Zidane       1.0000000


In [15]:
%%R
# Verificando a similaridade de Garrincha em relação a todos os outros usuários 
sim_users[,1]

      Garrincha      Pele Silva Romario Moreira Ronaldo Tavares      Zico Gomes 
      1.0000000       0.6628490       0.8934051       0.9912407       0.9244735 
Zinedine Zidane 
      0.3812464 


In [16]:
%%R
# Prevendo valores desconhecidos (filmes que poderão ser recomendados)
# Separando filmes ainda não avaliados
?setDT
rating_critic  = setDT(movie_ratings[colnames(movie_ratings)[1]],keep.rownames = TRUE)[]
names(rating_critic) = c('title','rating')

In [17]:
%%R
# Isolando filmes ainda não avaliados
titles_na_critic = rating_critic$title[is.na(rating_critic$rating)]
ratings_t = ratings[ratings$title %in% titles_na_critic,]
View(ratings_t)

R[write to console]: Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In View(ratings_t) :
R[write to console]:  unable to open display




Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View


In [18]:
%%R
# Adicionando valores de similaridade para cada usuÃ¡rio como uma nova variável
x = (setDT(data.frame(sim_users[,6]),keep.rownames = TRUE)[])
names(x) = c('critic','similarity')
ratings_t =  merge(x = ratings_t, y = x, by = "critic", all.x = TRUE)
View(ratings_t)

R[write to console]: Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In View(ratings_t) :
R[write to console]:  unable to open display




Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View


In [19]:
%%R
# Multiplicando ratings e valores de similaridade
ratings_t$sim_rating = ratings_t$rating * ratings_t$similarity
View(ratings_t)

R[write to console]: Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In View(ratings_t) :
R[write to console]:  unable to open display




Error in .External2(C_dataviewer, x, title) : unable to start data viewer
Calls: <Anonymous> -> <Anonymous> -> withVisible -> View


In [20]:
%%R
# Prevendo títulos ainda não avaliados
result = ratings_t %>% group_by(title) %>% summarise(sum(sim_rating)/sum(similarity))
result

R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



# A tibble: 3 x 2
  title              `sum(sim_rating)/sum(similarity)`
  <fct>                                          <dbl>
1 Just My Luck                                    2.34
2 Lady in the Water                               2.90
3 The Night Listener                              3.35


In [21]:
%%R
# Mas qual dos 3 filmes recomendar primeiro? 
# Aquele com score maior que a média de ratings do usuário Garrincha
mean(rating_critic $ rating, na.rm = T)

[1] 3.166667


In [26]:
%%R
# Função para sistema de recomendação (une os itens realizados nos comandos anteriores) 
generateRecommendations <- function(userId){
  rating_critic  = setDT(movie_ratings[colnames(movie_ratings)[userId]],keep.rownames = TRUE)[]
  names(rating_critic) = c('title','rating')
  titles_na_critic = rating_critic$title[is.na(rating_critic$rating)]
  ratings_t =ratings[ratings$title %in% titles_na_critic,]
  x = (setDT(data.frame(sim_users[,userId]),keep.rownames = TRUE)[])
  names(x) = c('critic','similarity')
  ratings_t =  merge(x = ratings_t, y = x, by = "critic", all.x = TRUE)
  ratings_t$sim_rating = ratings_t$rating*ratings_t$similarity
  result = ratings_t %>% group_by(title) %>% summarise(sum(sim_rating)/sum(similarity))
  return(result)
}

In [27]:
%%R
# Gerando recomendações
generateRecommendations(1)
generateRecommendations(2)
generateRecommendations(3)
generateRecommendations(5)


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)

R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)

R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)

R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



# A tibble: 0 x 2
# … with 2 variables: title <fct>, `sum(sim_rating)/sum(similarity)` <dbl>
